### II - Récupération des données foncières géolocalisées 

à partir de l'API DVF+ Géomutations, nous pouvons récupérer l'ensemble des transactions foncières ainsi que leurs valeur foncière sur une periode donnée. Nous nous concentrerons sur les données pour le département du Gard. 



## Récupération des transactions foncières dans le Gard à partir de l'API DVF+ Open Data 

l'API impose une contrainte de taille maximale d'emprise de 0.02° x 0.02° via le paramètre in_bbox. Ainsi, pour déterminer l'ensemble des transactions dans le département du Gard, il est nécessaire d'automatiser le processus d'interrogation de L'API en suivant les étapes : 
1. Obtenir les limites géographiques du Gard : Identifiez les coordonnées minimales et maximales en latitude et longitude du département.
2. Diviser le Gard en plusieurs petites emprises qui quadrilleront l'ensemble du territoire 
3. Automatiser les requêtes pour chaque in_bbox 
4. Pre processing des données : Récupération des transactions pour les types de logements : Appartement ou maison, gestion des doublons à partir de la clé 'id' 
5. Visualisation 



In [ ]:
#1 

from cartiflette import carti_download

# Télécharger les données du Gard via Cartiflette
gard_data = carti_download(
    crs=4326,  # Coordonnées géographiques (latitude/longitude)
    values="30",  # Code du département du Gard
    borders="COMMUNE",  # Niveau administratif : Communes
    vectorfile_format="geojson",  # Format de sortie
    filter_by="DEPARTEMENT",  # Filtrer par département
    source="EXPRESS-COG-CARTO-TERRITOIRE",  # Source des données
    year=2022  # Année des données
)

# Calculer les limites du Gard
bounds = gard_data.total_bounds  # Renvoie [longitude_min, latitude_min, longitude_max, latitude_max]

# Afficher les limites
print("Limites géographiques du Gard (bounding box) :")
print(f"Longitude min : {bounds[0]}")
print(f"Latitude min : {bounds[1]}")
print(f"Longitude max : {bounds[2]}")
print(f"Latitude max : {bounds[3]}")


In [ ]:
#2

import geopandas as gpd
from shapely.geometry import Polygon
from cartiflette import carti_download

# Fonction pour générer des plages de coordonnées
def frange(start, stop, step):
    """Génère une plage de valeurs avec un pas donné."""
    values = []
    while start < stop:
        values.append(round(start, 6))  # Limite les décimales pour éviter les erreurs d'arrondi
        start += step
    return values

# Télécharger les bordures du Gard via Cartiflette
gard_data = carti_download(
    crs=4326,  # Coordonnées géographiques (latitude/longitude)
    values="30",  # Code du département du Gard
    borders="COMMUNE",  # Niveau administratif : Communes
    vectorfile_format="geojson",  # Format de sortie
    filter_by="DEPARTEMENT",  # Filtrer par département
    source="EXPRESS-COG-CARTO-TERRITOIRE",  # Source des données
    year=2022  # Année des données
)


# Limites géographiques du Gard (bounding box)
xmin, ymin, xmax, ymax = gard_data.total_bounds

# Taille d'une cellule (0.02° x 0.02°)
cell_size = 0.02

# Liste pour stocker les polygones
grid_cells = []

# Générer le quadrillage brut
x_coords = frange(xmin, xmax, cell_size)  # Générer les longitudes
y_coords = frange(ymin, ymax, cell_size)  # Générer les latitudes

for x in x_coords:
    for y in y_coords:
        # Créer un polygone pour chaque cellule
        grid_cells.append(Polygon([
            (x, y),
            (x + cell_size, y),
            (x + cell_size, y + cell_size),
            (x, y + cell_size),
            (x, y)
        ]))

# Créer un GeoDataFrame avec le quadrillage brut
grid_gdf = gpd.GeoDataFrame(grid_cells, columns=["geometry"], crs="EPSG:4326")

# Découper le quadrillage avec les contours du Gard
grid_clipped = gpd.overlay(grid_gdf, gard_data, how="intersection")

print("Quadrillage adapté au Gard généré")


In [ ]:
#Visualisation du quadrillage 

import folium
import geopandas as gpd
from cartiflette import carti_download


# Définir le centre de la carte (approximativement au centre du Gard)
center_lat, center_lon = (44.0, 4.1)  # Coordonnées centrales du Gard

# Créer une carte Folium
m = folium.Map(location=[center_lat, center_lon], zoom_start=9, tiles="cartodbpositron")

# Ajouter les bordures du Gard à la carte
folium.GeoJson(
    gard_data,
    style_function=lambda x: {
        "color": "black",       # Couleur des bordures
        "weight": 2,            # Épaisseur des bordures
        "fillOpacity": 0        # Pas de remplissage
    },
    name="Bordures du Gard"
).add_to(m)

# Ajouter le quadrillage à la carte
folium.GeoJson(
    grid_clipped,
    style_function=lambda x: {
        "color": "blue",       # Couleur des bordures
        "weight": 1,           # Épaisseur des bordures
        "fillColor": "yellow", # Couleur de remplissage
        "fillOpacity": 0.2,    # Transparence
    },
    name="Quadrillage du Gard"
).add_to(m)

# Ajouter une couche de contrôle
folium.LayerControl().add_to(m)

# Sauvegarder la carte dans un fichier HTML

m

NE PAS FAIRE TOURNER CETTE CELLULE CI DESSOUS, TEMPS D'EXECUTION : +10H 

In [ ]:
#3 Automatisation de l'interrogation de l'API pour l'ensemble du Gard 

import requests
import geopandas as gpd
import pandas as pd
from shapely.geometry import box

# Charger le quadrillage adapté
grid_clipped = gpd.read_file("gard_quadrillage_adapte.geojson")

# URL de base de l'API
BASE_URL = "https://apidf-preprod.cerema.fr/dvf_opendata/geomutations/"

# Fonction pour extraire les limites des bounding boxes (in_bbox)
def get_in_bbox(polygon):
    bounds = polygon.bounds  # [xmin, ymin, xmax, ymax]
    return f"{bounds[0]},{bounds[1]},{bounds[2]},{bounds[3]}"

# Fonction pour interroger l’API de manière récursive
def fetch_all_data_for_bbox(base_url, in_bbox):
    all_data = []  # Stockage des résultats pour cette cellule
    current_url = f"{base_url}?in_bbox={in_bbox}"  # URL initiale pour la cellule
    
    while current_url:
        try:
            # Requête API
            response = requests.get(current_url)
            response.raise_for_status()  # Vérifie les erreurs HTTP
            data = response.json()  # Convertir la réponse en JSON
            
            # Ajouter les données actuelles
            if "features" in data:
                all_data.extend(data["features"])
            
            # Passer à l'URL suivante (si disponible)
            current_url = data.get("next")
            print(f"Page suivante : {current_url}")  # Afficher l'URL suivante
        except requests.exceptions.RequestException as e:
            print(f"Erreur pour {current_url} : {e}")
            break
    
    return all_data

# Parcourir chaque cellule du quadrillage et interroger l’API
all_data = []
for idx, row in grid_clipped.iterrows():
    in_bbox = get_in_bbox(row.geometry)  # Extraire le bounding box
    print(f"Traitement de la cellule {idx + 1}/{len(grid_clipped)} : {in_bbox}")
    
    # Récupérer toutes les données pour cette cellule
    cell_data = fetch_all_data_for_bbox(BASE_URL, in_bbox)
    
    if cell_data:
        all_data.extend(cell_data)  # Ajouter les données de cette cellule à la liste globale

# Convertir toutes les données en DataFrame
df = pd.json_normalize(all_data)

print("Toutes les données ont été sauvegardées dans 'transactions_dvf_gard_complete.csv'.")


Le fichier CSV renvoyé est trop volumineux pour les limites de stockage git. Nous filtrons donc au préalable la Dataframe avec les éléments qui nous intéressent afin de pouvoir la stocker pour réutilisation (sans avoir à ré-exécuter l'appel à l'API)

**Cette cellule n'est donc également pas à exécuter puisqu'elle prend en argument le fichier csv présupposément chargé après l'exécution de la cellule du dessus .**  

In [ ]:
#4. 

import pandas as pd
df = pd.read_csv('/home/onyxia/work/Python-For-Data-Science-Project/Shapefile processing/transactions_dvf_gard_complete.csv')
df = df.drop(['type','properties.idmutinvar', 'properties.idopendata','properties.coddep','properties.l_idpar','properties.l_idparmut','properties.l_idlocmut','properties.nbcomm','properties.nbpar','properties.nbparmut','properties.nbvolmut','properties.nblocmut','properties.vefa'], axis=1)
df = df[df["properties.codtypbien"].isin([121, 111])] # Conserver uniquement les transactions correspondant à des appartements ou maisons 
df = df.drop_duplicates(subset="id") # Retirer les doublons 
df.to_csv("transactions_dvf_gard_filtered.csv", index=False) 

Finalement, nous nous retrouvons avec le fichier 'transactions_dvf_gard_filtered.csv' comprenant les informations nécessaires sur les transactions immobilières dans le Gard. 
Le fichier csv a été enregistré en local puis renvoyée sous format Zip sur Git sous le nom 'transactions_dvf_gard_filtre' afin de respecter les limites de dépot git 

In [10]:
#Convertir 'transactions_dvf_gard_filtre' en geodataframe

import pandas as pd
import geopandas as gpd
import json
from shapely.geometry import shape

df = pd.read_csv('/home/onyxia/work/Python-For-Data-Science-Project/DVF Processing/transactions_dvf_gard_filtre.zip')

# Convertir les chaînes de caractères en listes JSON
df["geometry.coordinates"] = df["geometry.coordinates"].apply(json.loads) #ne pas charger si la cellule a déjà été exécutée une fois (le type a déjà été mis a jour)

# Créer des objets géométriques Shapely
df["geometry"] = df["geometry.coordinates"].apply(
    lambda coords: shape({"type": "MultiPolygon", "coordinates": coords})
)

gdf = gpd.GeoDataFrame(df, geometry="geometry", crs="EPSG:2154")  # CRS WGS84 pour Folium

#récupération des centroids uniquement, afin que l'affichage sur carte soit plus rapide, et plus ergonomique à la lecture (construction de cluster)
gdf["geometry"] = gdf["geometry"].centroid
gdf = gdf.drop(['geometry.type','geometry.coordinates'], axis =1)

In [12]:
#Récupérer le fichier geoJson des transactions immobilières

# Chemin de sortie pour le fichier Shapefile
shapefile_path = "gdf_trans_immo.geojson"

# Sauvegarder la GeoDataFrame
gdf.to_file(shapefile_path, driver="GeoJSON")


Comme Folium ne supporte pas le format Datestamp, nous nous sommes assurés que les éléments ne comprennent pas d'objets DateStamp

In [ ]:
#5 

import folium
from folium.plugins import MarkerCluster
import geopandas as gpd

# Créer une carte Folium
m = folium.Map(location=[43.83, 4.36], zoom_start=14, tiles="cartodbpositron")

# Créer un MarkerCluster
marker_cluster = MarkerCluster().add_to(m)

# Ajouter chaque point avec les informations détaillées dans la tooltip
for _, row in gdf.iterrows():
    # Vérifier que la géométrie est un Point
    if row.geometry.geom_type == "Point":
        # Extraire les informations pour la tooltip
        valeurfonc = row.get("properties.valeurfonc", "Non disponible")
        datemut = row.get("properties.datemut", "Non disponible")
        sbati = row.get("properties.sbati", "Non disponible")
        libtypbien = row.get("properties.libtypbien", "Non disponible")

        # Ajouter le marqueur avec la tooltip
        folium.Marker(
            location=[row.geometry.y, row.geometry.x],  # Latitude et Longitude
            tooltip=(
                f"<b>Valeur foncière :</b> {valeurfonc}<br>"
                f"<b>Date de transaction :</b> {datemut}<br>"
                f"<b>Nombre de m² :</b> {sbati}<br>"
                f"<b>Type de propriété :</b> {libtypbien}"
            )
        ).add_to(marker_cluster)

# Ajouter une couche de contrôle
folium.LayerControl().add_to(m)

# Sauvegarder la carte dans un fichier HTML
m.save("centroids_marker_cluster_map.html")
print("Carte générée et sauvegardée dans 'centroids_marker_cluster_map.html'.")

# Afficher la carte
m


Cette carte interactive met en évidence l'ensemble des transactions immobilières disponibles à partir de l'API DVF+ Open Data. Chaque transaction foncière est caractérisée par : 
- ID
- Valeur Foncière
- Date de transaction 
- Type de logement